In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!cd /usr/local/lib/python3.8/dist-packages/pyspark/jars && wget https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/1.2.20.1043/RedshiftJDBC42-no-awssdk-1.2.20.1043.jar

--2023-01-23 23:03:42--  https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/1.2.20.1043/RedshiftJDBC42-no-awssdk-1.2.20.1043.jar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.1.46, 52.216.92.229, 54.231.164.168, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.1.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2413910 (2.3M) [application/java-archive]
Saving to: ‘RedshiftJDBC42-no-awssdk-1.2.20.1043.jar.1’

RedshiftJDBC42-no-a 100%[===================>]   2.30M  11.5MB/s    in 0.2s    

2023-01-23 23:03:42 (11.5 MB/s) - ‘RedshiftJDBC42-no-awssdk-1.2.20.1043.jar.1’ saved [2413910/2413910]



In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL #1") \
    .getOrCreate()

In [2]:
rows_test = [
    { 'value': 1, 'name': 'Luka' },
    { 'value': 2, 'name': 'Luka'},
    { 'value': 3, 'name': 'Dirk' },
    { 'value': 4, 'name': 'Dirk' },
    { 'value': 5, 'name': 'Luka' },
]

df = spark.createDataFrame(rows_test)

In [3]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- value: long (nullable = true)



In [4]:
df.show()

+----+-----+
|name|value|
+----+-----+
|Luka|    1|
|Luka|    2|
|Dirk|    3|
|Dirk|    4|
|Luka|    5|
+----+-----+



In [5]:
df.createOrReplaceTempView("rows_test")

In [6]:
spark.sql("""
  SELECT 
    value,
    SUM(value) OVER (
        order by value 
        rows between 2 preceding and 2 following
    ) AS rolling_sum
  FROM rows_test""").show()

+-----+-----------+
|value|rolling_sum|
+-----+-----------+
|    1|          6|
|    2|         10|
|    3|         15|
|    4|         14|
|    5|         12|
+-----+-----------+



In [7]:
spark.sql("""
  SELECT 
    value,
    SUM(value) OVER (
        order by value 
        rows between unbounded preceding and 2 following
    ) AS rolling_sum
  FROM rows_test""").show()

+-----+-----------+
|value|rolling_sum|
+-----+-----------+
|    1|          6|
|    2|         10|
|    3|         15|
|    4|         15|
|    5|         15|
+-----+-----------+



In [8]:
# rows between은 FIRST_VALUE, LAST_VALUE 사용할 때는 항상 들어간다.
# 특정 범위내에서 할 것인지, 전체 범위 내에서 진행할 것인지
# unbounded를 쓰면 특정 윈도우 내에서 활용
spark.sql("""
  SELECT 
    *,
    FIRST_VALUE(value) OVER (
        partition by name
        order by value 
        rows between unbounded preceding and unbounded following
    ) AS min_value,
    LAST_VALUE(value) OVER (
        partition by name
        order by value 
        rows between unbounded preceding and unbounded following
    ) AS max_value    
  FROM rows_test""").show()

+----+-----+---------+---------+
|name|value|min_value|max_value|
+----+-----+---------+---------+
|Dirk|    3|        3|        4|
|Dirk|    4|        3|        4|
|Luka|    1|        1|        5|
|Luka|    2|        1|        5|
|Luka|    5|        1|        5|
+----+-----+---------+---------+



## Redshift 상의 다음 테이블을 데이터프레임으로 로딩하기
user_session_channel, session_timestamp, session_transaction

In [9]:
# Redshift와 연결해서 DataFrame으로 로딩하기
url = "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234"

df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()

df_session_transaction = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_transaction") \
    .load()

In [10]:
df_user_session_channel.createOrReplaceTempView("user_session_channel")
df_session_timestamp.createOrReplaceTempView("session_timestamp")
df_session_transaction.createOrReplaceTempView("session_transaction")

In [11]:
df_user_session_channel.show(5)

+------+--------------------+-------+
|userid|           sessionid|channel|
+------+--------------------+-------+
|  1491|00029153d12ae1c9a...|Organic|
|    59|0002ac0d783338cfe...|  Naver|
|   117|0006246bee639c7a7...|Youtube|
|   572|0006dd05ea1e999dd...|Organic|
|   935|0007cda84fafdcf42...| Google|
+------+--------------------+-------+
only showing top 5 rows



In [12]:
df_session_timestamp.show(5)

+--------------------+--------------------+
|           sessionid|                  ts|
+--------------------+--------------------+
|00029153d12ae1c9a...|2019-10-18 14:14:...|
|0004289ee1c7b8b08...|2019-11-16 21:20:...|
|0006246bee639c7a7...|2019-08-10 16:33:...|
|0006dd05ea1e999dd...|2019-07-06 19:54:...|
|000958fdaefe0dd06...|2019-11-02 14:52:...|
+--------------------+--------------------+
only showing top 5 rows



In [15]:
df_session_transaction.show(5)

+--------------------+--------+------+
|           sessionid|refunded|amount|
+--------------------+--------+------+
|00029153d12ae1c9a...|   false|    85|
|008909bd27b680698...|   false|    13|
|0107acb41ef20db22...|   false|    16|
|018544a2c48077d2c...|   false|    39|
|020c38173caff0203...|   false|    61|
+--------------------+--------+------+
only showing top 5 rows



## 위의 테이블들을 이용해서 사용자별로 처음 채널과 마지막 채널 알아내기

In [13]:
# WITH를 활용해서 임시 테이블 만들기
# 임시 테이블 해서 ROW_NUMBER를 부르는데, 정렬을 다르게 한 후
# 채널을 구하기 위해서 SELF 조인형태처럼 진행

first_last_channel_df = spark.sql("""
WITH RECORD AS (
  SELECT /*사용자의 유입에 따른, 채널 순서 매기는 쿼리*/
      userid,
      channel, 
      ROW_NUMBER() OVER (PARTITION BY userid ORDER BY ts ASC) AS seq_first,
      ROW_NUMBER() OVER (PARTITION BY userid ORDER BY ts DESC) AS seq_last
  FROM user_session_channel u
  LEFT JOIN session_timestamp t
    ON u.sessionid = t.sessionid
)
SELECT /*유저의 첫번째 유입채널, 마지막 유입 채널 구하기*/
      f.userid,
      f.channel first_channel,
      l.channel last_channel
FROM RECORD f
INNER JOIN RECORD l ON f.userid = l.userid
WHERE f.seq_first = 1 and l.seq_last = 1
ORDER BY userid
""")

In [14]:
first_last_channel_df.show(10)

+------+-------------+------------+
|userid|first_channel|last_channel|
+------+-------------+------------+
|    27|      Youtube|   Instagram|
|    29|        Naver|       Naver|
|    33|       Google|     Youtube|
|    34|      Youtube|       Naver|
|    36|        Naver|     Youtube|
|    40|      Youtube|      Google|
|    41|     Facebook|     Youtube|
|    44|        Naver|   Instagram|
|    45|      Youtube|   Instagram|
|    59|    Instagram|   Instagram|
+------+-------------+------------+
only showing top 10 rows



In [20]:
# BOUND 한 것 중에 PARTITION 하고 같은 PARTITION 안에서 오름 차순으로 세팅을 한 다음에
# 채널의 FIRST VALUE, LAST VALUE를 뽑는다

first_last_channel_df2 = spark.sql("""
SELECT DISTINCT A.userid,
    FIRST_VALUE(A.channel) over(partition by A.userid order by B.ts
rows between unbounded preceding and unbounded following) AS First_Channel,
    LAST_VALUE(A.channel) over(partition by A.userid order by B.ts
rows between unbounded preceding and unbounded following) AS Last_Channel
FROM user_session_channel A
LEFT JOIN session_timestamp B
ON A.sessionid = B.sessionid""")

In [21]:
first_last_channel_df2.show()

+------+-------------+------------+
|userid|First_Channel|Last_Channel|
+------+-------------+------------+
|    27|      Youtube|   Instagram|
|    29|        Naver|       Naver|
|    33|       Google|     Youtube|
|    34|      Youtube|       Naver|
|    36|        Naver|     Youtube|
|    40|      Youtube|      Google|
|    41|     Facebook|     Youtube|
|    44|        Naver|   Instagram|
|    45|      Youtube|   Instagram|
|    59|    Instagram|   Instagram|
|    64|      Youtube|     Youtube|
|    65|      Youtube|     Organic|
|    68|      Youtube|     Organic|
|    69|     Facebook|   Instagram|
|    80|      Organic|       Naver|
|    84|       Google|     Youtube|
|    87|      Youtube|      Google|
|    97|      Organic|     Organic|
|   112|     Facebook|     Youtube|
|   113|      Organic|     Organic|
+------+-------------+------------+
only showing top 20 rows

